In [ ]:
import tkinter as tk
from tkinter import filedialog
import subprocess
import os



def browse_files():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_paths = filedialog.askopenfilenames(filetypes=[("LAZ files", "*.laz")])
    return list(file_paths)

# Function to create a unique file path by adding a version number
def create_unique_filepath(base_filepath):
    if not os.path.exists(base_filepath):
        return base_filepath

    base, ext = os.path.splitext(base_filepath)
    version = 2

    while os.path.exists(f"{base}_v{version}{ext}"):
        version += 1

    return f"{base}_v{version}{ext}"

def run_sub_samp():
    exe = r"C:\Program Files\CloudCompare\CloudCompare.exe"
    assert os.path.exists(exe), 'CloudCompare must be installed to run this app.'

    input_files = browse_files()
    all_flt_folders = [os.path.dirname(os.path.dirname(file)) for file in input_files]
    all_flt_folders_are_the_same = all([file == all_flt_folders[0] for file in all_flt_folders])
    assert all_flt_folders_are_the_same, 'All laz files should come from the same flight folder.'

    output_file_name = 'SubSamp_Merge_' + os.path.basename(all_flt_folders[0])+'.laz'
    output_file_path = os.path.join(os.path.dirname(input_files[0]), output_file_name)

    output_file_path = create_unique_filepath(output_file_path)

    # '''
    # "C:\Program Files\CloudCompare\CloudCompare.exe" -SILENT -O "C:\Users\olivi\Desktop\VW data processing\surface point cloud whole area\wolverine surface point cloud whole area.laz" -SS SPATIAL 1.45 -C_EXPORT_FMT LAS -EXT .laz -SAVE_CLOUDS FILE "C:\Users\olivi\Desktop\VW data processing\surface point cloud whole area\wolverine surface point cloud whole area sub sampled.laz"
    
    
    # "C:\Program Files\CloudCompare\CloudCompare.exe" -SILENT      -O "E:\TEMP_DRONE_STUFF_V2\To2Fl1-L-2024-05-12-13-12-16\GeoreferenceResult\laser_1_48404.625_48544.525_001637___WITH_TOWER.laz"            -O "E:\TEMP_DRONE_STUFF_V2\To2Fl1-L-2024-05-12-13-12-16\GeoreferenceResult\laser_1_48252.925_48391.835_001620.laz"    -SS SPATIAL 2   -MERGE_CLOUDS  -SOR   6    2.0         -C_EXPORT_FMT LAS -EXT .laz -SAVE_CLOUDS FILE "E:\TEMP_DRONE_STUFF_V2\To2Fl1-L-2024-05-12-13-12-16\GeoreferenceResult\TWO_LINES_SOR6-2_subed2_MERGED.laz"
    # '''
    

    # Start building the command line
    cmd_line = [
        exe,
        '-SILENT', '-AUTO_SAVE', 'OFF',
        ]
    for file in input_files:
        cmd_line.extend(['-O', file])

    cmd_line.extend(['-SS', 'SPATIAL', '0.75', '-MERGE_CLOUDS', '-SOR', '6', '2.0',
                     '-C_EXPORT_FMT', 'LAS', '-EXT', '.laz', '-SAVE_CLOUDS', 'FILE', output_file_path])

    print('cmd_line inuput:', ' '.join(cmd_line))
    # Run the command
    process = subprocess.Popen(cmd_line, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # Read and print output line by line
    for line in process.stdout:
        print(line, end='')
    # Read and print errors line by line
    for line in process.stderr:
        print(line, end='')

    print('Done')


if __name__ == '__main__':
    run_sub_samp()